In [ ]:
from nbdev import *
%nbdev_default_export functions
#%nbdev_default_class_level 3

Cells will be exported to pct.functions,
unless a different module is specified after an export flag: `%nbdev_export special.module`


In [ ]:
%nbdev_hide
%reload_ext autoreload
%autoreload 2

# Functions

> Functions that form the elements of a perceptual control node (system).


## Overview
Each function outputs the result of applying the function logic and may be a scalar or array, depending on the input. The inputs are supplied as links to another function, from where the values are read. Unless the function is a simple case, such as a 'Constant'. 

Parameters are supplied as arguments to the constructor of the class.

In [ ]:
%nbdev_export
import numpy as np
from abc import ABC, abstractmethod

In [ ]:
%nbdev_export
class UniqueNamer:
    __instance = None
    @staticmethod 
    def getInstance():
      """ Static access method. """
      if UniqueNamer.__instance == None:
         UniqueNamer()
      return UniqueNamer.__instance
    def __init__(self):
      """ Virtually private constructor. """
      if UniqueNamer.__instance != None:
         raise Exception("This class is a singleton!")
      else:
         UniqueNamer.__instance = self
      self.names = {}

    def clear(self):
      self.names = {}

    def get_name(self, name):
        if name in self.names.keys(): 
            num = self.names[name]+1
            self.names[name]=num
            name = f'{name}{num}'
        else:
            self.names[name]=0
        return name

In [ ]:
%nbdev_export
class FunctionsList:
    __instance = None
    @staticmethod 
    def getInstance():
      """ Static access method. """
      if FunctionsList.__instance == None:
         FunctionsList()
      return FunctionsList.__instance
    def __init__(self):
      """ Virtually private constructor. """
      if FunctionsList.__instance != None:
         raise Exception("This class is a singleton!")
      else:
         FunctionsList.__instance = self
      self.functions = {}

    def clear(self):
      self.functions = {}

    
    def add_function(self, func):
        name = func.get_name()
        self.functions[name]=func
        
        return name
    
    def get_function(self, name):
        func = self.functions[name]
        
        return func

In [ ]:
%nbdev_export
class BaseFunction(ABC):
    "Base class of a PCT function. This class is not used direclty by developers, but defines the functionality common to all."
    def __init__(self, name, value):
        self.value = value
        self.links = []
        self.name = UniqueNamer.getInstance().get_name(name)
        FunctionsList.getInstance().add_function(self)
        
    @abstractmethod
    def __call__(self, verbose=False):
        if verbose :
            print(f'{self.value:.3f}', end= " ")
            
        return self.value
    
    @abstractmethod    
    def summary(self, str):
        print(f'{self.name} {type(self).__name__}', end = " ")
        if len(str)>0:
            print(f'| {str}', end= " ")
        print(f'| {self.value}', end = " ")
        if len(self.links)>0:
            print(f'| links ', end=" ")
        for link in self.links:
            print(link.get_name(), end= " ")
        print()
        
    @abstractmethod    
    def get_config(self):
        config = {"type": type(self).__name__,
                    "name": self.name, 
                    "value": self.value}
        
        ctr=0
        links={}
        for link in self.links:
            links[ctr]=link.get_name()
            ctr+=1
        
        config['links']=links
        return config
        
    def get_name(self):
        return self.name
    
    def set_name(self, name):
        self.name=name

    def set_value(self, value):
        self.value= value
    
    def get_value(self):
        return self.value
    
    def add_link(self, linkfn):
        self.links.append(linkfn)
    
    @classmethod
    def from_config(cls, config):
        func = cls(**config)
        key  = 'links'
        if key in config:
            for key in config['links'].keys():
                func.links.append(FunctionsList.getInstance().get_function(config['links'][key]))
        return func


In [ ]:
%nbdev_export
class Proportional(BaseFunction):
    "A proportion of the input value as defined by the gain parameter. Parameters: The gain value. Links: One."
    def __init__(self, gain=1, value=0, name="proportional", **cargs):
        super().__init__(name, value)
        self.gain = gain

    def __call__(self, verbose=False):
        input = self.links[0].get_value()
        self.value = input * self.gain
        return super().__call__(verbose)
    
    def summary(self):
        super().summary(f'gain {self.gain}')

    def get_config(self):
        config = super().get_config()
        config["gain"] = self.gain
        return config               

In [ ]:
%nbdev_export
class Variable(BaseFunction):
    "A function that returns a variable value. Parameter: The variable value. Links: None"
    def __init__(self,  value=0, name="variable", **cargs):
        super().__init__(name, value)
    
    def __call__(self, verbose=False):
        return super().__call__(verbose)
    
    def summary(self):
        super().summary("")
        
    def get_config(self):
        config = super().get_config()
        return config



In [ ]:
%nbdev_export
class Subtract(BaseFunction):
    "A function that subtracts one value from another. Parameter: None. Links: Two links required to each the values to be subtracted."
    def __init__(self, value=0, name="subtract", **cargs):
        super().__init__(name, value)
    
    def __call__(self, verbose=False):
        #print("Sub ", self.links[0].get_value(),self.links[1].get_value() )
        self.value = self.links[0].get_value()-self.links[1].get_value()

        return super().__call__(verbose)

    def summary(self):
        super().summary("")

    def get_config(self):
        return super().get_config()
                        

In [ ]:
%nbdev_export
class Constant(BaseFunction):
    "A function that returns a constant value. Parameter: The constant value. Links: None"
    def __init__(self, value=0, name="constant", **cargs):
        super().__init__(name, value)
    
    def __call__(self, verbose=False):
        return super().__call__(verbose)
    
    def summary(self):
        super().summary("")

    def get_config(self):
        return super().get_config()


In [ ]:
%nbdev_export
class Integration(BaseFunction):
    "A leaky integrating function. Equivalent of a exponential smoothing function, of the amplified input. Parameter: The gain and slow values. Links: One."
    def __init__(self, gain=1, slow=2, value=0, name="integration", **cargs):
        super().__init__(name, value)
        self.gain = gain
        self.slow = slow
    
    def __call__(self, verbose=False):
        input = self.links[0].get_value()
        self.value = self.value +  ((input * self.gain) - self.value)/self.slow
        
        return super().__call__(verbose)

    def summary(self):
        super().summary(f'gain {self.gain} slow {self.slow} ')

    def get_config(self):
        config = super().get_config()
        config["gain"] = self.gain
        config["slow"] = self.slow
        return config       
        

In [ ]:
%nbdev_export
class WeightedSum(BaseFunction):
    "A function that combines a set of inputs by multyplying each by a weight and then adding them up. Parameter: The weights array. Links: Links to all the input functions."
    def __init__(self, value=0, weights=np.ones(3), name="weighted_sum", **cargs):
        super().__init__(name, value)
        self.weights = weights
    
    def __call__(self, verbose=False):
        if len(self.links) != self.weights.size:
            raise Exception(f'Number of links {len(self.links)} and weights {self.weights.size} must be the same.')
        
        inputs = np.array([link.get_value() for link in self.links])
        self.value = np.dot(inputs, self.weights)

        return super().__call__(verbose)

    def summary(self):
        super().summary("")

    def get_config(self):
        config = super().get_config()
        config["weights"] = self.weights
        return config

## Creating Functions

Standard class constructor. Different ways to create a function with the standard constructor.

In [ ]:
prop = Proportional()
print(prop.get_config())
prop = Proportional("myprop", 10)
print(prop.get_config())
prop = Proportional(gain=10)
print(prop.get_config())

{'type': 'Proportional', 'name': 'proportional', 'value': 0, 'links': {}, 'gain': 1}
{'type': 'Proportional', 'name': 'proportional1', 'value': 10, 'links': {}, 'gain': 'myprop'}
{'type': 'Proportional', 'name': 'proportional2', 'value': 0, 'links': {}, 'gain': 10}


Configuration class constructor. Create the function by passing a configuration structure to the constructor.

In [ ]:
prop = Proportional(**{'name': 'myprop', 'value': 5, 'gain': 20})
print(prop.get_config())

{'type': 'Proportional', 'name': 'myprop', 'value': 5, 'links': {}, 'gain': 20}


Configuration class method. Create the function by passing a configuration structure to a class method.

In [ ]:
config = {'name': 'myprop', 'value': -0.5, 'gain': 21}
prop = Proportional.from_config(config)
print(prop.get_config())

{'type': 'Proportional', 'name': 'myprop1', 'value': -0.5, 'links': {}, 'gain': 21}


An example showing creating a WeightedSum function.

In [ ]:
wts=np.ones(3)
ws = WeightedSum(weights=wts)
ws.add_link(Constant(10))
ws.add_link(Constant(5))
ws.add_link(Constant(20))
assert ws() == 35
ws.get_config()

{'type': 'WeightedSum',
 'name': 'weighted_sum',
 'value': 35.0,
 'links': {0: 'constant', 1: 'constant1', 2: 'constant2'},
 'weights': array([1., 1., 1.])}

In [ ]:
ws1 = WeightedSum.from_config(ws.get_config())
ws1.get_config()

{'type': 'WeightedSum',
 'name': 'weighted_sum1',
 'value': 35.0,
 'links': {0: 'constant', 1: 'constant1', 2: 'constant2'},
 'weights': array([1., 1., 1.])}

## Viewing Functions

View the details of the function with the "summary", which prints the name, type, parameters, value and links (if any).

In [ ]:
prop.summary()

myprop1 Proportional | gain 21 | -0.5 


As already seen the function details can be seen by retrieving the configuration.

In [ ]:
print(prop.get_config())

{'type': 'Proportional', 'name': 'myprop1', 'value': -0.5, 'links': {}, 'gain': 21}


# Setting Links
The next cell shows how a link is added to one function from another. In this case from an Integration function to a Constant function. So, whenever "integrator" runs it will get its input from "cons". 

In [ ]:
integrator = Integration(3, 10)
cons = Constant(5)
integrator.add_link(cons)
integrator.summary()

integration Integration | gain 3 slow 10  | 0 | links  constant3 


In [ ]:
#print(integrator.get_config())
inte = Integration.from_config(integrator.get_config())
print(inte())
print(inte.get_config())
target = {'type': 'Integration', 'name': 'integration1', 'value': 1.5, 'links': {0: 'constant3'}, 'gain': 3, 'slow': 10}
print(target)
assert inte.get_config() == target

1.5
{'type': 'Integration', 'name': 'integration1', 'value': 1.5, 'links': {0: 'constant3'}, 'gain': 3, 'slow': 10}
{'type': 'Integration', 'name': 'integration1', 'value': 1.5, 'links': {0: 'constant3'}, 'gain': 3, 'slow': 10}


## Running a Fucntion
A function can simply be run by calling it, without any parameters. It will use whatever input was set by the links. It returns the result of the function. In this example it will be 5 \* 3 / 10, that is, input \* gain / slow.

In [ ]:
output = integrator()
print(output)
assert output == 1.5

1.5


## Array Input
The input to a function can be an array rather than a single value.

In [ ]:
integrator.set_value(np.array([1, 2, 4, 3]))
output = integrator()
print(output)
np.testing.assert_array_equal(output, [2.4, 3.3, 5.1, 4.2])

[2.4 3.3 5.1 4.2]


In [ ]:
print(integrator.get_config())

{'type': 'Integration', 'name': 'integration', 'value': array([2.4, 3.3, 5.1, 4.2]), 'links': {0: 'constant3'}, 'gain': 3, 'slow': 10}


In [ ]:
#print(UniqueNamer.getInstance().names)
UniqueNamer.getInstance().clear()
integ = Integration(**{'name': 'myinteg', 'value': 1, 'gain': 20, 'slow': 100})
prop = Proportional(5, name="myprop")
integ.add_link(prop)
print(integ.get_config())
assert integ.get_config() == {'type': 'Integration', 'name': 'myinteg', 'value': 1, 'links': {0: 'myprop'}, 'gain': 20, 'slow': 100}

{'type': 'Integration', 'name': 'myinteg', 'value': 1, 'links': {0: 'myprop'}, 'gain': 20, 'slow': 100}


In [ ]:
#hide
from nbdev import *
notebook2script()

Converted 00_core.ipynb.
Converted 01_rmath.ipynb.
Converted 02_functions.ipynb.
Converted 03_nodes.ipynb.
Converted 04_hierarchy.ipynb.
Converted index.ipynb.
